In [ ]:
// #r ".\binaries2\bossspad.dll"
// #r ".\binaries2\XNSEC.dll"
// #r "C:\BoSSS_Binaries\bossspad.dll"
// #r "C:\BoSSS_Binaries\XNSEC.dll"
#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

## Case configuration

In [ ]:
int nCells = 8;
int dg = 2;
// double[] multiplierS = new double[] {1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0};
// double[] multiplierS = new double[] {1.0,2.0};
// int[] multiplierS = new int[]{1,4,6,8,10,14};
int[] multiplierS = new int[]{1,2,3,4,5,6,7,8,9,10,11,12,13,14};

// int[] multiplierS = new int[]{4};

double InitialMassFuelIn = 0.02400; //kg/m2s 
double InitialMassAirIn = 0.02400 *3; //kg/m2s 
bool parabolicVelocityProfile = false;
bool chemicalReactionActive = true; 
int numberOfMpiCores = 8;
bool useFullGeometry = false;
bool wallBounded = true;
bool AMRinEachNewtonIteration = true;

## Open Database

In [ ]:
static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\CDF_HomotopyTest");
// static var myDb = OpenOrCreateDatabase(@"C:\Databases\CDF_NoCombustionStrainSweep_");
// myDb.Summary()

In [ ]:
BoSSSshell.WorkflowMgm.Init("CounterFlowFlame2");
// BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.Sessions


In [ ]:
var myBatch = BoSSSshell.ExecutionQueues[3];
// MiniBatchProcessor.Server.StartIfNotRunning(true);
myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDb.Path,""));

In [ ]:
1+1

2

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid(int nCells, bool wallBounded) {
        double L = 0.02;
        double LRef = L;
        double R_dim = L / 2;
        double separation = 1.0; // nondimensional
        double xleft = 0;
        double xright = separation;
        double radius_inlet = R_dim / LRef;
        double ybot = -separation * 3; 
        double ytop = separation * 3; 

        // Define Grid
        double[] _xNodes;
        double[] _yNodes;

        // X-NODES
        double xNodesDensity = 6.0; 
        _xNodes = GenericBlas.Linspace(xleft, xright, (int)(nCells * xNodesDensity + 1));
        // Y-NODES
        double sf3 = 0.80;
        var _yNodes2 = GenericBlas.Linspace(-radius_inlet, radius_inlet, nCells*2 + 1).ToList(); // center
        Console.WriteLine(_yNodes2);
        List<double> yNodesTop = (GenericBlas.SinLinSpacing(radius_inlet, (ytop - radius_inlet) * 2 + radius_inlet, sf3, nCells * 2 + 1).ToList()); // Nodes corresponding to the oxidizer inlet, right part
        var myYnodesTop = yNodesTop.GetSubVector(0, yNodesTop.Count / 2 + 1); // Take only "bottom side" of node array
        var myYNodesBot = myYnodesTop.CloneAs();
        myYNodesBot.ScaleV(-1.0);
        Array.Reverse(myYNodesBot);
        List<double> list2 = new List<double>();
        list2.AddRange(myYNodesBot.Take(myYNodesBot.Length - 1).ToList());
        list2.AddRange(_yNodes2.Take(_yNodes2.Count() - 1).ToList());
        list2.AddRange(myYnodesTop.ToList());
        _yNodes = list2.ToArray();

        var grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes);

        // Define Edge tags
        grd.EdgeTagNames.Add(1, "Velocity_Inlet_CH4");
        grd.EdgeTagNames.Add(2, "Velocity_Inlet_O2");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");

        if(wallBounded){
            grd.EdgeTagNames.Add(4, "Wall");
        }
        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];
            //Edge tags
            //1: Velocity inlet O_2
            //2: Velocity inlet CH_4
            //3: Pressure outlet
            if(Math.Abs(x - xleft) < 1e-8) { // Left boundary
                if(Math.Abs(y) - radius_inlet < 1e-8) { // Fuel Inlet
                    return 1;
                } else {
                    if(wallBounded){
                    return 4;
                } else{
                    return 3;
                }
                }
            }
            if(Math.Abs(x - xright) < 1e-8) { // right boundary
                if(Math.Abs(y) - radius_inlet < 1e-8) { // oxy Inlet
                    return 2;//2
                } else {
                    if(wallBounded){
                        return 4;
                    } else{
                        return 3;
                    }
                }
            } else {
                return 3; //3 Pressure outlet
            }
        });
        myDb.SaveGrid(ref grd, true);

        return grd;
    }

    public static Grid2D GenerateGrid_FullGeometry(int nCells) {
        double L = 0.02;
        double LRef = L;
        double R_dim = L / 2;
        double separation = 1.0; // nondimensional
        double xleft = 0;
        double xright = separation;
        double radius_inlet = R_dim / LRef;
        double ybot = -separation * 3; //
        double ytop = separation * 3;//

        // Define Grid
        double[] _xNodes;
        double[] _yNodes;

         // X-NODES
        double[] XNODES1 = GenericBlas.Linspace(0, separation, (int)(nCells / 2 + 1));
        double[] XNODES2 = GenericBlas.Linspace(separation * 1, separation * 2, (int)(nCells + 1));
        double[] XNODES3 = GenericBlas.Linspace(separation * 2, separation * 3, (int)(nCells / 2 + 1));
        List<double> xNodesList = new List<double>();
        xNodesList.AddRange(XNODES1.Take(XNODES1.Length - 1));
        xNodesList.AddRange(XNODES2.Take(XNODES2.Length - 1));
        xNodesList.AddRange(XNODES3);
        _xNodes = xNodesList.ToArray();

        // Y-NODES
        double sf3 = 0.80;
        var _yNodes2 = GenericBlas.Linspace(-radius_inlet, radius_inlet, nCells + 1).ToList(); // center
        List<double> yNodesTop = (GenericBlas.SinLinSpacing(radius_inlet, (ytop - radius_inlet) * 2 + radius_inlet, sf3, nCells * 2 + 1).ToList()); // Nodes corresponding to the oxidizer inlet, right part
        var myYnodesTop = yNodesTop.GetSubVector(0, yNodesTop.Count / 2 + 1); // Take only "bottom side" of node array
        var myYNodesBot = myYnodesTop.CloneAs();
        myYNodesBot.ScaleV(-1.0);
        Array.Reverse(myYNodesBot);
        List<double> list2 = new List<double>();
        list2.AddRange(myYNodesBot.Take(myYNodesBot.Length - 1).ToList());
        list2.AddRange(_yNodes2.Take(_yNodes2.Count() - 1).ToList());
        list2.AddRange(myYnodesTop.ToList());
        _yNodes = list2.ToArray();

        double[] CutOut1Point1 = new double[2] { separation, separation / 2 };
        double[] CutOut1Point2 = new double[2] { 0, -separation / 2 };

        double[] CutOut2Point1 = new double[2] { 2 * separation, -separation / 2 };
        double[] CutOut2Point2 = new double[2] { 3 * separation, separation / 2 };

        var CutOut1 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut1.AddPoint(CutOut1Point1);
        CutOut1.AddPoint(CutOut1Point2);

        var CutOut2 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut2.AddPoint(CutOut2Point1);
        CutOut2.AddPoint(CutOut2Point2);
        var CutOuts = new BoSSS.Platform.Utils.Geom.BoundingBox[] { CutOut1, CutOut2 };
        var grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes, CutOuts: CutOuts);

        grd.EdgeTagNames.Add(1, "Velocity_Inlet_CH4");
        grd.EdgeTagNames.Add(2, "Velocity_Inlet_O2");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");
        grd.EdgeTagNames.Add(4, "Wall");

        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];
            //Edge tags
            //1: Velocity inlet O_2
            //2: Velocity inlet CH_4
            //3: Pressure outlet
            if(Math.Abs(x - separation) < 1e-8 && Math.Abs(y) - radius_inlet < 1e-8) {
                return 1;
            }
            if(Math.Abs(x - 2 * separation) < 1e-8 && Math.Abs(y) - radius_inlet < 1e-8) {
                return 2;
            }
            if(Math.Abs(y - radius_inlet) < 1e-8 && x > 0 && x < separation) {
                return 4;
            }
            if(Math.Abs(y - radius_inlet) < 1e-8 && x > 2 * separation && x < 3 * separation) {
                return 4;
            }
            if(Math.Abs(y + radius_inlet) < 1e-8 && x > 0 && x < separation) {
                return 4;
            }
            if(Math.Abs(y + radius_inlet) < 1e-8 && x > 2 * separation && x < 3 * separation) {
                return 4;
            }
            return 3;
        }
        );
        myDb.SaveGrid(ref grd, true);

        return grd;
    }
}

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double ConstVal, double inletRadius, double uInFuel, double uInAir) {
        using(var stw = new System.IO.StringWriter()) {
           
           stw.WriteLine("static class BoundaryValues {");

           stw.WriteLine("  static public double ConstantValue(double[] X) {");
           stw.WriteLine("    return "+ ConstVal +";");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double ParabolaVelocityFuel(double[] X) {");
           stw.WriteLine("    return  (1.0 - Math.Pow(X[1] / "+inletRadius+", 2)) * "+uInFuel+" ;");
           stw.WriteLine("  }");
           
           stw.WriteLine("  static public double ParabolaVelocityAir(double[] X) {");
           stw.WriteLine("    return  -(1.0 - Math.Pow(X[1] / "+inletRadius+", 2)) * "+uInAir+";");
           stw.WriteLine("  }");
           
       
            stw.WriteLine("}"); 
           return stw.ToString();
        }
    }

    static public Formula Get_ConstantValue(double ConstVal, double inletRadius, double uInFuel, double uInAir){
        return new Formula("BoundaryValues.ConstantValue", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir));
    }

    static public Formula Get_ParabolaVelocityFuel(double ConstVal, double inletRadius, double uInFuel, double uInAir){
        return new Formula("BoundaryValues.ParabolaVelocityFuel", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir));
    }
    static public Formula Get_ParabolaVelocityAir(double ConstVal, double inletRadius, double uInFuel, double uInAir){
        return new Formula("BoundaryValues.ParabolaVelocityAir", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir));
    }
}

A base control file, where basic configuration of the CounterDiffusionFlame is defined.

In [ ]:
static XNSEC_Control GiveMeTheCtrlFile(int dg, int nCells, bool isMF, double massFuelIn, double massAirIn, bool parabolicVelocityProfile, bool UsefullGeometry, bool wallBounded) {
    var CC = new ChemicalConstants();

    var C = isMF ? new XNSEC_MF_Control() : new XNSEC_Control();

    C.NumberOfChemicalSpecies = 4; 
    C.SetDGdegree(dg); //
    if(UsefullGeometry) {
        C.SetGrid(GridFactory.GenerateGrid_FullGeometry(nCells));  //
    } else {
        C.SetGrid(GridFactory.GenerateGrid(nCells, wallBounded));  //
    }

    C.MatParamsMode = MaterialParamsMode.Sutherland;  //
    // Problem Definition
    //===================
    double L = 0.02; // separation between the two inlets, meters
    double TemperatureInFuel = 300;  //
    double TemperatureInOxidizer = 300; //
    double AtmPressure = 101325; // Pa
    double[] FuelInletConcentrations = new double[] { 0.2, 0.0, 0.0, 0.0, 0.8 };
    double[] OxidizerInletConcentrations = new double[] { 0.0, 0.23, 0.0, 0.0, 0.77 }; //
    double[] MWs = new double[] { CC.MW_CH4, CC.MW_O2, CC.MW_CO2, CC.MW_H2O, CC.MW_N2 };

    double mwFuel = CC.getAvgMW(MWs, FuelInletConcentrations);
    double mwAir = CC.getAvgMW(MWs, OxidizerInletConcentrations);
    double densityAirIn = AtmPressure * mwAir / (CC.R_gas * TemperatureInOxidizer * 1000); // kg / m3
    double densityFuelIn = AtmPressure * mwFuel / (CC.R_gas * TemperatureInFuel * 1000); // kg / m3.
    double uInFuel = massFuelIn / densityFuelIn; //
    double uInAir = massAirIn / densityAirIn; //

    Console.WriteLine("VelocityFuel" + uInFuel);
    Console.WriteLine("VelocityAir" + uInAir);

    // Reference values
    //===================
    // Basic units to be used: Kg, m, s, mol, pa,
    double TRef = TemperatureInOxidizer;// Reference temperature  is the inlet temperature, (K)
    double pRef = AtmPressure; // Pa
    double uRef = Math.Max(uInFuel, uInAir); // m/s
    double LRef = L;
    C.GravityDirection = new double[] { 0.0, 0.0, 0.0 }; //No gravity.

    // Solver configuration
    // =======================

    C.NonLinearSolver.ConvergenceCriterion = 1e-8;
    C.LinearSolver.ConvergenceCriterion = 1e-10;

    C.NonLinearSolver.verbose = true;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.MaxSolverIterations = 10;
    C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();

    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    C.saveperiod = 1;
    C.PenaltyViscMomentum = 1.0;
    C.PenaltyHeatConduction = 1.0;
    C.YFuelInlet = FuelInletConcentrations[0];
    C.YOxInlet = OxidizerInletConcentrations[1];
    C.FuelInletConcentrations = FuelInletConcentrations;
    C.OxidizerInletConcentrations = OxidizerInletConcentrations;
    C.TFuelInlet = 1.0;
    C.TOxInlet = 1.0;
    C.PhysicalParameters.IncludeConvection = true;
    // Chemical related parameters
    double s = (CC.nu_O2 * CC.MW_O2) / (CC.nu_CH4 * CC.MW_CH4);
    C.phi = s * C.YFuelInlet / C.YOxInlet;
    C.zSt = 1.0 / (1.0 + C.phi);
    var MLC = new MaterialLawCombustion(300, new double[] { }, C.MatParamsMode, C.rhoOne, true, 1.0, 1, 1, C.YOxInlet, C.YFuelInlet, C.zSt, CC, 0.75);
    var ThermoProperties = new ThermodynamicalProperties();



    bool easymode = false;

    if (easymode){
    C.rhoOne = true;
    C.cpOne = true;
    C.MatParamsMode = MaterialParamsMode.Constant; 

    } 






    //==========================
    //Derived reference values
    //==========================
    C.uRef = uRef; // Reference velocity
    C.LRef = LRef; // reference length
    C.pRef = AtmPressure; // reference pressure
    C.TRef = TemperatureInFuel;// reference temperature
    C.MWRef = MLC.getAvgMW(MWs, C.OxidizerInletConcentrations); // Air mean molecular weight
    C.rhoRef = C.pRef * C.MWRef / (8.314 * C.TRef * 1000); // Kg/m3. ok ;
    C.cpRef = 1.3;//ThermoProperties.Calculate_Cp_Mixture(new double[] { 0.23, 0.77 }, new string[] { "O2", "N2" }, 300); // 1.219185317353029;// Representative value, KJ/Kg K ========> 1.31 for the one-step kinetic model
    C.muRef = MLC.getViscosityDim(300);
    C.MolarMasses = new double[] { C.CC.MW_CH4, C.CC.MW_O2, C.CC.MW_CO2, C.CC.MW_H2O, C.CC.MW_N2 };
    C.MolarMasses.ScaleV(1.0 / C.MWRef); //NonDimensionalized Molar masses

    double heatRelease_Ref = (C.TRef * C.cpRef);
    C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref;
    C.B = CC.PreExponentialFactor;

    C.StoichiometricCoefficients = new double[] { -1, -2, 1, 2, 0 };

    C.Damk = C.rhoRef * C.LRef * C.B / (C.uRef * C.MWRef);
    C.Reynolds = C.rhoRef * C.uRef * C.LRef / C.muRef;
    C.Prandtl = 1.0;//0.75;////////////////////0.75;
    C.Schmidt = C.Prandtl; // Because Lewis number  is assumed as 1.0  (Le = Pr/Sc)
    // C.Lewis = new double[] { 0.97, 1.11, 1.39, 0.83, 1.0 };
    C.Lewis = new double[] { 1.0, 1.0, 1.0, 1.0, 1.0 };

    double g = 9.8; // m/s2
    C.Froude = Math.Sqrt(uRef * uRef / (C.LRef * g)); // Not used
    C.T_ref_Sutherland = 300; //////// Check this
    C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 }; // NOTE! activation temperature is also nondimensional
    //==========================
    // Initial conditions
    //==========================
    double Radius = 0.5;
    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uInFuel / C.uRef, uInAir / C.uRef));
    C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uInFuel / C.uRef, uInAir / C.uRef));
    C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uInFuel / C.uRef, uInAir / C.uRef));
    //==========================
    // Boundary conditions
    //==========================
    double dummy = 0;
    if(parabolicVelocityProfile) {
        C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ParabolaVelocityFuel(dummy, Radius, uInFuel / C.uRef, dummy));
        C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ParabolaVelocityAir(dummy, Radius, dummy, uInAir / C.uRef));
    } else { // Plug Flow
        C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(uInFuel / C.uRef, dummy, dummy, dummy));
        C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue((-1) * uInAir / C.uRef, dummy, dummy, dummy));
    }

    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy));
    if(wallBounded){
        C.AddBoundaryValue("Wall", VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy));
        C.AddBoundaryValue("Wall", VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy));
    }
    return C;
}

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static XNSEC_Control GiveMeTheMixtureFractionCtrlFile(int dg, int nCells, double massFuelIn, double massAirIn, bool parabolicVelocityProfile, int multiplier, bool useFullGeometry, bool wallBounded){
    var C_MixtureFraction = GiveMeTheCtrlFile(dg, nCells, true,massFuelIn, massAirIn, parabolicVelocityProfile, useFullGeometry,  wallBounded );
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.ProjectName = "CounterDifFlame";
    string name = C_MixtureFraction.ProjectName + "P" + dg + "K" + nCells + "Multiplier" +multiplier;
    C_MixtureFraction.SessionName = "FS_" + name;

    C_MixtureFraction.UseSelfMadeTemporalOperator = false;
    C_MixtureFraction.ChemicalReactionActive = false;
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.NonLinearSolver.MaxSolverIterations = 500; 
    C_MixtureFraction.dummycounter = multiplier;
    // Boundary and initial conditions
    double dummy = -11111111;
    C_MixtureFraction.AddInitialValue(VariableNames.MixtureFraction,BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy));
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy));
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(0.0,dummy,dummy , dummy));

    C_MixtureFraction.Tags.Add(multiplier.ToString());
    double radius_inlet = 0.5; // radius is 1/2 from length separation
    var troubledPoints = new List<double[]>();
    if(useFullGeometry){
        troubledPoints.Add(new double[]{2, +radius_inlet });
        troubledPoints.Add(new double[]{2, -radius_inlet });
        troubledPoints.Add(new double[]{1, +radius_inlet });
        troubledPoints.Add(new double[]{1, -radius_inlet });
    } else {
        troubledPoints.Add(new double[]{0, +radius_inlet });
        troubledPoints.Add(new double[]{0, -radius_inlet });
        troubledPoints.Add(new double[]{1, +radius_inlet });
        troubledPoints.Add(new double[]{1, -radius_inlet });
    }
    
    C_MixtureFraction.AdaptiveMeshRefinement = true;
    C_MixtureFraction.AMR_startUpSweeps = 2;
    int NoOfPseudoTimesteps = 3;
    C_MixtureFraction.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady; 
    C_MixtureFraction.NoOfTimesteps = NoOfPseudoTimesteps ;

    bool useHomotopy = true;
    double desiredVelMultiplier = 10;
    if(useHomotopy) { 
        C_MixtureFraction.HomotopyApproach = XNSEC_Control.HomotopyType.Automatic;
        C_MixtureFraction.HomotopyVariable = XNSEC_Control.HomotopyVariableEnum.VelocityInletMultiplier;
        C_MixtureFraction.homotopieAimedValue = desiredVelMultiplier;
    }

    C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onProblematicPoints(troubledPoints,C_MixtureFraction.AMR_startUpSweeps) );  
    C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onFlameSheet(C_MixtureFraction.zSt,NoOfPseudoTimesteps) );
    return C_MixtureFraction;
}

## Send and run jobs

In [ ]:
    double massFuelIn = InitialMassFuelIn;
    double massAirIn = InitialMassAirIn;

    Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
    var C_MixtureFraction = GiveMeTheMixtureFractionCtrlFile(dg, nCells, massFuelIn, massAirIn,  parabolicVelocityProfile,1,useFullGeometry, wallBounded);
    string jobName       = C_MixtureFraction.SessionName+"_finer";
    Console.WriteLine(jobName);
    var oneJob           = new Job(jobName, solver_MF);
    oneJob.NumberOfMPIProcs = 8;
    oneJob.NumberOfThreads = 1;
    oneJob.SetControlObject(C_MixtureFraction);
    oneJob.Activate(myBatch); 


System.Collections.Generic.List`1[System.Double]


Grid Edge Tags changed.


VelocityFuel0.02426418525959607


VelocityAir0.06147806069578092


FS_CounterDifFlameP2K8Multiplier1_finer


...

Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\CounterFlowFlame2-XNSEC2021Aug04_165245


copied 52 files.


   written file: control.obj


   copied 'amd64' runtime.


deployment finished.


In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

## Starting the finite-rate chemistry simulation

Now that the simulation for an "infinite" reaction rate is done, we use it for initializing the system with finite reaction rate. The goal is to obtain solutions of the counter difussion flame for increasing strain values. We start with a low strain (bigger Dahmkoehler number), which is increased until extintion is (hopefully) found

In [ ]:
static XNSEC_Control GiveMeTheFullCtrlFile(int dg, int nCells, double massFuelIn, double massAirIn, ISessionInfo SessionToRestart, bool parabolicVelocityProfile, bool chemReactionActive, bool useFullGeometry, bool wallBounded, int mult, int counter) {
    var C_OneStep = GiveMeTheCtrlFile(dg, nCells, false, massFuelIn, massAirIn, parabolicVelocityProfile, useFullGeometry, wallBounded);
    C_OneStep.physicsMode = PhysicsMode.Combustion;
    C_OneStep.ProjectName = "CounterDifFlame";
    string name = C_OneStep.ProjectName + "P" + dg + "K" + nCells + "mult" + mult+"_c"+counter ;
    C_OneStep.SessionName = "Full_" + name;
    C_OneStep.VariableOneStepParameters = false;

    // C_OneStep.Tags.Add("VelocityMultiplier" + mult);
    C_OneStep.Tags.Add( mult.ToString());
    C_OneStep.cpOne = false;
    C_OneStep.dummycounter = counter;
    C_OneStep.UseSelfMadeTemporalOperator = false;
    C_OneStep.myThermalWallType = SIPDiffusionTemperature.ThermalWallType.Adiabatic;
    C_OneStep.Timestepper_LevelSetHandling = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
    C_OneStep.UseMixtureFractionsForCombustionInitialization = true;

    C_OneStep.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
    C_OneStep.LinearSolver.NoOfMultigridLevels = 5;
    C_OneStep.ChemicalReactionActive = chemReactionActive;
    C_OneStep.AdaptiveMeshRefinement = true;

    bool AMRinEachNewtonStep = true;

    if( AMRinEachNewtonStep) {
        C_OneStep.NoOfTimesteps = 20;
        C_OneStep.NonLinearSolver.MaxSolverIterations = 1; // Do only one newton iteration before refining
        C_OneStep.NonLinearSolver.MinSolverIterations = 1; // Do only one newton iteration before refining
    } else{
        C_OneStep.NoOfTimesteps = 1; // The steady solution will be calculated again and do AMR
        C_OneStep.NonLinearSolver.MaxSolverIterations = 20;
    }
   
    C_OneStep.AMR_startUpSweeps = 1;
    if(C_OneStep.ChemicalReactionActive){
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_onReactiveZones(C_OneStep.MolarMasses, 3, 0.7));
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnFieldGradient(3, 0.75, VariableNames.Temperature));
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 4));
    }
    // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 3));
    // C_OneStep.NonLinearSolver.MaxSolverIterations = 10;

    // Boundary conditions

    double dummy = 0;
    if(SessionToRestart != null) {
        C_OneStep.SetRestart(SessionToRestart);
    } else {
        C_OneStep.AddInitialValue(VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(0.23, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy));
    }
    if(wallBounded){
        C_OneStep.AddBoundaryValue("Wall", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy));
    }
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[0], dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[1], dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[2], dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[3], dummy, dummy, dummy));

    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[0], dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[1], dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[2], dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[3], dummy, dummy, dummy));
    return C_OneStep;
}

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);

Calculate the full solution for the initial value

In [ ]:
int counter = 0;
 
    var sess = myDb.Sessions[0];



In [ ]:
var C = GiveMeTheFullCtrlFile(dg,nCells,InitialMassFuelIn*5,InitialMassAirIn*5, sess , parabolicVelocityProfile, chemicalReactionActive,useFullGeometry, wallBounded,5, counter );
string jobName       = C.SessionName ;
Console.WriteLine(jobName);
var oneJob           = new Job(jobName, solver);
oneJob.NumberOfMPIProcs = numberOfMpiCores;    
oneJob.NumberOfThreads = 1;
oneJob.SetControlObject(C);
oneJob.Activate(myBatch); 
counter++;


System.Collections.Generic.List`1[System.Double]


Grid Edge Tags changed.


VelocityFuel0.12132092629798034


VelocityAir0.3073903034789046


Full_CounterDifFlameP2K8mult5_c0


...

Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\CounterFlowFlame2-XNSEC2021Aug04_162557


copied 52 files.


   written file: control.obj


   copied 'amd64' runtime.


deployment finished.


In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

In [ ]:
BoSSSshell.WorkflowMgm

Gradually increase  inlet velocity

In [ ]:
myDb.Sessions[0].Export().Do()

Starting export process... 

Data will be written to the following directory:


C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\CounterFlowFlame2__FS_CounterDifFlameP2K8Multiplier1__e5591adc-0895-4a70-804e-fea066f9318a

In [ ]:
myDb.Sessions

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs

In [ ]:
myDb.Sessions[0].Export().Do()

Starting export process... 

Data will be written to the following directory:


C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\CounterFlowFlame2__FS_CounterDifFlameP2K8Multiplier1__e5591adc-0895-4a70-804e-fea066f9318a

## Postprocessing

In [ ]:
 var sess = myDb.Sessions[13];
 var timestep_FullChem = sess.Timesteps.Last();

double eps = 1e-4; 
double xleft  = 0+eps; 
double xright = 1.0-eps;
int nCells = 181*1;
double[]  _xNodes = GenericBlas.Linspace(xleft, xright, nCells + 1);
string[] varnames = new string[]{"VelocityX","VelocityY","Temperature", "Pressure", "MassFraction0","MassFraction1","MassFraction2","MassFraction3"}; 


In [ ]:
Dictionary<string, double[]> Results_FullComb = new Dictionary<string, double[]>();
Dictionary<string, double> ReferenceValuesDict = new Dictionary<string, double>();// Re-dimenzionalise variables

ReferenceValuesDict.Add("VelocityX", Convert.ToDouble(sess.KeysAndQueries["uRef"]));
ReferenceValuesDict.Add("VelocityY", Convert.ToDouble(sess.KeysAndQueries["uRef"]));
ReferenceValuesDict.Add("Temperature", Convert.ToDouble(sess.KeysAndQueries["TRef"]));
ReferenceValuesDict.Add("Pressure", Convert.ToDouble(sess.KeysAndQueries["pRef"]));
ReferenceValuesDict.Add("MassFraction0", 1.0);
ReferenceValuesDict.Add("MassFraction1", 1.0);
ReferenceValuesDict.Add("MassFraction2", 1.0);
ReferenceValuesDict.Add("MassFraction3", 1.0);
ReferenceValuesDict.Add("MassFraction4", 1.0);
ReferenceValuesDict.Add("Length", Convert.ToDouble(sess.KeysAndQueries["LRef"]));

In [ ]:
//Save results from the Full problem simulation
foreach(var varname in varnames){
    double[] Yvalues = new double[nCells+1];
    var field = timestep_FullChem.Fields.Where( Fi => (Fi.Identification == varname)).Last();
    for(int n = 0 ; n < _xNodes.Length; n++){
        double pickedVal = field.ProbeAt(new double[]{ _xNodes[n], 0.00000001});    // Pick value along the central axis        
        Yvalues[n] = pickedVal* ReferenceValuesDict[varname];             // Re.dimenzionalise varliables and store it in array
    }
    Results_FullComb.Add(varname,Yvalues);
}

In [ ]:
// Redimensionalise the x coordinate
double[] _xNodesDim = new double[_xNodes.Length];
 for(int n = 0 ; n < _xNodes.Length; n++){
     _xNodesDim[n] = _xNodes[n]*ReferenceValuesDict["Length"];
}


In [ ]:
 string path = System.IO.Path.ChangeExtension(@"C:\BoSSS\experimental\internal\src\private-gutz\L4-application\XNSEC2\Worksheets\Jupyter\CounterFlowFlameMatlabCode\BoSSS_data\", null);
string newPathName = path +"Full" + "XCoord" + ".txt";
using (StreamWriter s = new StreamWriter(newPathName)) {
    for (int i = 0; i < _xNodes.Length; i++) {
        s.Write(_xNodesDim[i].ToString("E16", System.Globalization.NumberFormatInfo.InvariantInfo) + " ");
    }
}

foreach (var varname in varnames) {
    string newPathName = path +"Full" + varname + ".txt";
    using (StreamWriter s = new StreamWriter(newPathName)) {
        double[] _yNodes = Results_FullComb[varname];
        for (int i = 0; i < _xNodes.Length; i++) { 
            s.Write(_yNodes[i].ToString("E16", System.Globalization.NumberFormatInfo.InvariantInfo) + " "); 
        }
    }
}

In [ ]:
myDb

In [ ]:
OpenOrCreateDatabase(@"C:\Databases\BoSSS_DB")